# Exploratory Data Analysis of Teams dataset

## Section: Data loading

This section focuses on loading the teams dataset for further analysis and processing. 
- import ``teams``
- see subset of dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import pointbiserialr

teams = pd.read_csv('basketballPlayoffs/teams.csv', delimiter=",")

teams.head()

## Section: Shift Target Variable Function Implementation

This section introduces a function, `shift_target_variable`, designed to shift the target variable 'playoff' in a dataset. 

(We are no longer using this function because the shift is all done inside the main file)

In [ ]:
def shift_target_variable(dataset):
    dataset.sort_values(by=['tmID', 'year'], inplace=True)

    dataset['playoffs'] = dataset.groupby('tmID')['playoff'].shift(-1)

    dataset.drop(columns=['playoff'], inplace=True)

    dataset.dropna(subset=['playoffs'], inplace=True)

    dataset.rename(columns={'playoffs': 'playoff'}, inplace=True)

    dataset.reset_index(drop=True, inplace=True)

    return dataset

In [ ]:
#teams = shift_target_variable(teams)

## Section: Exploratory Data Analysis for ``teams`` dataset
This section involves exploratory data analysis on the `teams` dataset, providing insights into its structure and relationships. The analysis is divided into several subsections.


### See number of unique values in each column to verify which columns can have repeated data and which not
- Iterate through each column in the 'teams' DataFrame.
- Print the number of different values in each column.
- Visualize the results using a horizontal bar chart.

In [ ]:
for column in teams.columns:
    unique_values = teams[column].unique()
    print(f"Number of different values in the {column} column are:", len(unique_values))
    print("------------")

In [ ]:
columns = []
value_counts = []

for column in teams.columns:
    unique_values = teams[column].nunique()
    columns.append(column)
    value_counts.append(unique_values)

plt.figure(figsize=(10, 10))
plt.barh(columns, value_counts, color='skyblue')
plt.xlabel('Number of Unique Values')
plt.ylabel('Columns')
plt.title('Number of Unique Values in Each Column')
plt.show()


### See if there are missing values
- Check for missing values in the dataset.

In [ ]:
teams.isnull().sum()

### See what are the data types of each column
- Display data types of each column.

In [ ]:
print(teams.dtypes)

### Data Description 
- Create a copy of the original dataset for preservation.
- Convert categorical columns to numeric using category codes.
- Display descriptive statistics of the transformed dataset using the ``describe()`` function

In [ ]:
teams_original = teams.copy()

for column in teams.columns:
    if teams[column].dtype == 'object':
        teams[column] = teams[column].astype('category').cat.codes

teams.describe()

### Correlation Matrix
- Generate a correlation matrix for continuous variables.
- Visualize the matrix using a heatmap.
- Print correlations between the target variable (``playoff``) and other features.

In [ ]:
def correlation_matrix(dataframe):
    corr_matrix = dataframe.corr()

    target_correlation = corr_matrix['playoff']

    plt.figure(figsize=(30, 20))

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    sns.heatmap(corr_matrix, mask=mask, annot=True, annot_kws={"size": 8}, cmap='coolwarm', linewidths=0.5, fmt=".2f")

    plt.title('Correlation Matrix', fontsize=16)
    plt.show()

    dict = {}

    for feature, correlation in target_correlation.items():
        print(f"Correlation between target and {feature}: {correlation}")
        dict[feature] = correlation

    return dict

In [ ]:
correlation_matrix(teams)

### Chi-square Test
- Perform the chi-square test for categorical variables against the target variable (``playoff``).
- Print chi-square values, p-values, and test results.

In [ ]:
def chi_square(dataset, target): 
    alpha = 0.05
    
    for feature in dataset.columns:
        if feature != target:
            contingency_table = pd.crosstab(dataset[feature], dataset[target])

            # check if any category has no data
            if contingency_table.shape[0] == 0 or contingency_table.shape[1] == 0:
                print(f"No data for {feature} and {target}")
                continue
            
            chi2, p, observed, expected = chi2_contingency(contingency_table)
            
            # Step 4: Print or store the results
            print(f"Chi-square test for {feature} and {target}:")
            print(f"Chi-square value: {chi2}")
            print(f"P-value: {p}")
            if p < alpha:
                print("Reject the null hypothesis. There is a significant association.")
            else:
                print("Fail to reject the null hypothesis. There is not enough evidence for a significant association.")
            print("")

In [ ]:
chi_square(teams, 'playoff')

### Point-Biserial Correlation
- Calculate point-biserial correlation for continuous and categorical variables against the target variable (``playoff``).
- Display correlation coefficients and p-values in a DataFrame.

In [ ]:
# continuous variables
def point_biserial(dataset, features, target):
    binary_target = dataset[target]

    # Create an empty dictionary to store correlation coefficients and p-values
    correlation_results = {}

    # Iterate through each column in the DataFrame
    for column in features:
            # Calculate point-biserial correlation
            correlation, p_value = pointbiserialr(dataset[column], binary_target)
            
            # Store the results in the dictionary
            correlation_results[column] = {'correlation': correlation, 'p_value': p_value}

    # Convert the dictionary to a DataFrame for easier analysis
    results_df = pd.DataFrame.from_dict(correlation_results, orient='index')

    # Display the results
    print(results_df)

In [ ]:
continuous_features = ["year", "rank", "o_fgm","o_fga","o_ftm","o_fta","o_3pm","o_3pa","o_oreb","o_dreb","o_reb","o_asts","o_pf","o_stl","o_to","o_blk","o_pts","d_fgm","d_fga","d_ftm","d_fta","d_3pm","d_3pa","d_oreb","d_dreb","d_reb","d_asts","d_pf","d_stl","d_to","d_blk","d_pts","tmORB","tmDRB","tmTRB","opptmORB","opptmDRB","opptmTRB","won","lost","GP","homeW","homeL","awayW","awayL","confW","confL","min","attend"]
categorical_features = ["tmID", "confID", "firstRound","semis","finals","arena"]
target = "playoff"

point_biserial(teams, continuous_features, 'playoff')

In [ ]:
point_biserial(teams, categorical_features, 'playoff')

- 0.00 to 0.19: Very weak correlation
- 0.20 to 0.39: Weak correlation
- 0.40 to 0.59: Moderate correlation
- 0.60 to 0.79: Strong correlation
- 0.80 to 1.00: Very strong correlation

Sometimes when comparing features with binary values we should not use Correlation Matrix. When the Correlation Matrix is applied to binary variables, it might not capture the full extent of the association, so in order to be sure we are taking the right conclusions we applied other tools in order to understand feature correlation. 

As we can see, features related to the playoffs are heavily related to whether the team made it to playoffs or not (as expected).  
Some features have no correlation, so they can be eliminated ('lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB')  
Some others like 'name', ' tmID' and 'franchID' are highly related to each other, which means we can delete and leave just one of them. 

In [ ]:
teams = teams.drop(columns=['name', 'franchID'])
teams_original = teams_original.drop(columns=['name', 'franchID'])

## Section: Calculate team power ranking

We can start by reducing the number of feature a team has and make them better features for evaluation, like Points-per-Game for example.

- Offensive Field Goal Percentage: Percentage of successful field goals.
- Offensive Free Throw Percentage: Percentage of successful free throws.
- Offensive Three-Point Percentage: Percentage of successful three-pointers.
- Offensive Rebounds Per Game: Average number of offensive rebounds per game.
- Offensive Assists Per Game: Average number of offensive assists per game.
- Offensive Personal Fouls Per Game: Average number of offensive personal fouls per game.
- Offensive Steals Per Game: Average number of offensive steals per game.
- Offensive Turnovers Per Game: Average number of offensive turnovers per game.
- Offensive Blocks Per Game: Average number of offensive blocks per game.
- Offensive Points Per Game: Average number of offensive points per game.

In [ ]:
# offensive stats
offensiveFieldGoalPercentageMax = 0
offensiveFieldGoalPercentageMin = 100

offensiveFreeThrowPercentageMax = 0
offensiveFreeThrowPercentageMin = 100

offensiveThreePointPercentageMax = 0
offensiveThreePointPercentageMin = 100

offensiveReboundsPerGameMax = 0
offensiveReboundsPerGameMin = 999

offensiveAssistsPerGameMax = 0
offensiveAssistsPerGameMin = 999

offensivePersonalFoulsPerGameMax = 0
offensivePersonalFoulsPerGameMin = 999

offensiveStealsPerGameMax = 0
offensiveStealsPerGameMin = 999

offensiveTurnoversPerGameMax = 0
offensiveTurnoversPerGameMin = 999

offensiveBlocksPerGameMax = 0
offensiveBlocksPerGameMin = 999

offensivePointsPerGameMax = 0
offensivePointsPerGameMin = 999

- Defensive Field Goal Percentage: Percentage of successful field goals allowed.
- Defensive Free Throw Percentage: Percentage of successful free throws allowed.
- Defensive Three-Point Percentage: Percentage of successful three-pointers allowed.
- Defensive Rebounds Per Game: Average number of defensive rebounds allowed per game.
- Defensive Assists Per Game: Average number of defensive assists allowed per game.
- Defensive Personal Fouls Per Game: Average number of defensive personal fouls allowed per game.
- Defensive Steals Per Game: Average number of defensive steals allowed per game.
- Defensive Turnovers Per Game: Average number of defensive turnovers allowed per game.
- Defensive Blocks Per Game: Average number of defensive blocks allowed per game.
- Defensive Points Per Game: Average number of defensive points allowed per game.

In [ ]:
# defensive stats
defensiveFieldGoalPercentageMax = 0
defensiveFieldGoalPercentageMin = 100

defensiveFreeThrowPercentageMax = 0
defensiveFreeThrowPercentageMin = 100

defensiveThreePointPercentageMax = 0
defensiveThreePointPercentageMin = 100

defensiveReboundsPerGameMax = 0
defensiveReboundsPerGameMin = 999

defensiveAssistsPerGameMax = 0
defensiveAssistsPerGameMin = 999

defensivePersonalFoulsPerGameMax = 0
defensivePersonalFoulsPerGameMin = 999

defensiveStealsPerGameMax = 0
defensiveStealsPerGameMin = 999

defensiveTurnoversPerGameMax = 0
defensiveTurnoversPerGameMin = 999

defensiveBlocksPerGameMax = 0
defensiveBlocksPerGameMin = 999

defensivePointsPerGameMax = 0
defensivePointsPerGameMin = 999

- Win Percentage: Percentage of games won.
- Home Win Percentage: Percentage of home games won.
- Away Win Percentage: Percentage of away games won.
- Conference Win Percentage: Percentage of conference games won.
- Attendance: Normalized attendance.

In [ ]:
# global stats
winPercentageMax = 0
winPercentageMin = 100

homeWinPercentageMax = 0
homeWinPercentageMin = 100

awayWinPercentageMax = 0
awayWinPercentageMin = 100

conferenceWinPercentageMax = 0
conferenceWinPercentageMin = 100

attendanceMax = 0
attendanceMin = 9999999

rankMax = 0
rankMin = 1

o_orebMax = 0
o_orebMin = 999

o_drebMax = 0
o_drebMin = 999

d_orebMax = 0
d_orebMin = 999

d_drebMax = 0
d_drebMin = 999

In this subsection, we iterate through each team in the dataset for a given year, calculating maximum and minimum values for various offensive, defensive, and global performance metrics. This preprocessing step is crucial for normalizing data before computing power rankings.

Key Metrics:
- Offensive and defensive percentages (e.g., field goals, free throws).
- Rebounds, assists, steals, turnovers, blocks, and points per game.
- Win percentages (total, home, away, conference).
- Attendance and team rank.

This process ensures a standardized scale for subsequent power ranking computations.

In [ ]:
yearly_stats = {}

In [ ]:
for year, year_data in teams.groupby('year'):
  
    year_dict = {}

    for index, row in year_data.iterrows():
        # offensive stats
        offensiveFieldGoalPercentage = (row['o_fgm'] / row['o_fga']) * 100

        # Update max and min for each stat per year
        year_dict.setdefault('offensiveFieldGoalPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveFieldGoalPercentage']['max'] = max(year_dict['offensiveFieldGoalPercentage']['max'], offensiveFieldGoalPercentage)
        year_dict['offensiveFieldGoalPercentage']['min'] = min(year_dict['offensiveFieldGoalPercentage']['min'], offensiveFieldGoalPercentage)

        offensiveFreeThrowPercentage = (row['o_ftm'] / row['o_fta']) * 100

        year_dict.setdefault('offensiveFreeThrowPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveFreeThrowPercentage']['max'] = max(year_dict['offensiveFreeThrowPercentage']['max'], offensiveFreeThrowPercentage)
        year_dict['offensiveFreeThrowPercentage']['min'] = min(year_dict['offensiveFreeThrowPercentage']['min'], offensiveFreeThrowPercentage)


        offensiveThreePointPercentage = (row['o_3pm'] / row['o_3pa']) * 100

        year_dict.setdefault('offensiveThreePointPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveThreePointPercentage']['max'] = max(year_dict['offensiveThreePointPercentage']['max'], offensiveThreePointPercentage)
        year_dict['offensiveThreePointPercentage']['min'] = min(year_dict['offensiveThreePointPercentage']['min'], offensiveThreePointPercentage)


        offensiveReboundsPerGame = row['o_reb'] / row['GP']

        year_dict.setdefault('offensiveReboundsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveReboundsPerGame']['max'] = max(year_dict['offensiveReboundsPerGame']['max'], offensiveReboundsPerGame)
        year_dict['offensiveReboundsPerGame']['min'] = min(year_dict['offensiveReboundsPerGame']['min'], offensiveReboundsPerGame)


        offensiveAssistsPerGame = row['o_asts'] / row['GP']

        year_dict.setdefault('offensiveAssistsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveAssistsPerGame']['max'] = max(year_dict['offensiveAssistsPerGame']['max'], offensiveAssistsPerGame)
        year_dict['offensiveAssistsPerGame']['min'] = min(year_dict['offensiveAssistsPerGame']['min'], offensiveAssistsPerGame)

        offensivePersonalFoulsPerGame = row['o_pf'] / row['GP']

        year_dict.setdefault('offensivePersonalFoulsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensivePersonalFoulsPerGame']['max'] = max(year_dict['offensivePersonalFoulsPerGame']['max'], offensivePersonalFoulsPerGame)
        year_dict['offensivePersonalFoulsPerGame']['min'] = min(year_dict['offensivePersonalFoulsPerGame']['min'], offensivePersonalFoulsPerGame)

        offensiveStealsPerGame = row['o_stl'] / row['GP']

        year_dict.setdefault('offensiveStealsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveStealsPerGame']['max'] = max(year_dict['offensiveStealsPerGame']['max'], offensiveStealsPerGame)
        year_dict['offensiveStealsPerGame']['min'] = min(year_dict['offensiveStealsPerGame']['min'], offensiveStealsPerGame)

        offensiveTurnoversPerGame = row['o_to'] / row['GP']

        year_dict.setdefault('offensiveTurnoversPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveTurnoversPerGame']['max'] = max(year_dict['offensiveTurnoversPerGame']['max'], offensiveTurnoversPerGame)
        year_dict['offensiveTurnoversPerGame']['min'] = min(year_dict['offensiveTurnoversPerGame']['min'], offensiveTurnoversPerGame)

        offensiveBlocksPerGame = row['o_blk'] / row['GP']

        year_dict.setdefault('offensiveBlocksPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensiveBlocksPerGame']['max'] = max(year_dict['offensiveBlocksPerGame']['max'], offensiveBlocksPerGame)
        year_dict['offensiveBlocksPerGame']['min'] = min(year_dict['offensiveBlocksPerGame']['min'], offensiveBlocksPerGame)

        offensivePointsPerGame = row['o_pts'] / row['GP']

        year_dict.setdefault('offensivePointsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['offensivePointsPerGame']['max'] = max(year_dict['offensivePointsPerGame']['max'], offensivePointsPerGame)
        year_dict['offensivePointsPerGame']['min'] = min(year_dict['offensivePointsPerGame']['min'], offensivePointsPerGame)

        # defensive stats
        defensiveFieldGoalPercentage = (row['d_fgm'] / row['d_fga']) * 100

        year_dict.setdefault('defensiveFieldGoalPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveFieldGoalPercentage']['max'] = max(year_dict['defensiveFieldGoalPercentage']['max'], defensiveFieldGoalPercentage)
        year_dict['defensiveFieldGoalPercentage']['min'] = min(year_dict['defensiveFieldGoalPercentage']['min'], defensiveFieldGoalPercentage)

        defensiveFreeThrowPercentage = (row['d_ftm'] / row['d_fta']) * 100

        year_dict.setdefault('defensiveFreeThrowPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveFreeThrowPercentage']['max'] = max(year_dict['defensiveFreeThrowPercentage']['max'], defensiveFreeThrowPercentage)
        year_dict['defensiveFreeThrowPercentage']['min'] = min(year_dict['defensiveFreeThrowPercentage']['min'], defensiveFreeThrowPercentage)

        defensiveThreePointPercentage = (row['d_3pm'] / row['d_3pa']) * 100

        year_dict.setdefault('defensiveThreePointPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveThreePointPercentage']['max'] = max(year_dict['defensiveThreePointPercentage']['max'], defensiveThreePointPercentage)
        year_dict['defensiveThreePointPercentage']['min'] = min(year_dict['defensiveThreePointPercentage']['min'], defensiveThreePointPercentage)

        defensiveReboundsPerGame = row['d_reb'] / row['GP']

        year_dict.setdefault('defensiveReboundsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveReboundsPerGame']['max'] = max(year_dict['defensiveReboundsPerGame']['max'], defensiveReboundsPerGame)
        year_dict['defensiveReboundsPerGame']['min'] = min(year_dict['defensiveReboundsPerGame']['min'], defensiveReboundsPerGame)

        defensiveAssistsPerGame = row['d_asts'] / row['GP']

        year_dict.setdefault('defensiveAssistsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveAssistsPerGame']['max'] = max(year_dict['defensiveAssistsPerGame']['max'], defensiveAssistsPerGame)
        year_dict['defensiveAssistsPerGame']['min'] = min(year_dict['defensiveAssistsPerGame']['min'], defensiveAssistsPerGame)

        defensivePersonalFoulsPerGame = row['d_pf'] / row['GP']

        year_dict.setdefault('defensivePersonalFoulsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensivePersonalFoulsPerGame']['max'] = max(year_dict['defensivePersonalFoulsPerGame']['max'], defensivePersonalFoulsPerGame)
        year_dict['defensivePersonalFoulsPerGame']['min'] = min(year_dict['defensivePersonalFoulsPerGame']['min'], defensivePersonalFoulsPerGame)

        defensiveStealsPerGame = row['d_stl'] / row['GP']

        year_dict.setdefault('defensiveStealsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveStealsPerGame']['max'] = max(year_dict['defensiveStealsPerGame']['max'], defensiveStealsPerGame)
        year_dict['defensiveStealsPerGame']['min'] = min(year_dict['defensiveStealsPerGame']['min'], defensiveStealsPerGame)

        defensiveTurnoversPerGame = row['d_to'] / row['GP']

        year_dict.setdefault('defensiveTurnoversPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveTurnoversPerGame']['max'] = max(year_dict['defensiveTurnoversPerGame']['max'], defensiveTurnoversPerGame)
        year_dict['defensiveTurnoversPerGame']['min'] = min(year_dict['defensiveTurnoversPerGame']['min'], defensiveTurnoversPerGame)

        defensiveBlocksPerGame = row['d_blk'] / row['GP']

        year_dict.setdefault('defensiveBlocksPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensiveBlocksPerGame']['max'] = max(year_dict['defensiveBlocksPerGame']['max'], defensiveBlocksPerGame)
        year_dict['defensiveBlocksPerGame']['min'] = min(year_dict['defensiveBlocksPerGame']['min'], defensiveBlocksPerGame)

        defensivePointsPerGame = row['d_pts'] / row['GP']

        year_dict.setdefault('defensivePointsPerGame', {'max': -float('inf'), 'min': float('inf')})
        year_dict['defensivePointsPerGame']['max'] = max(year_dict['defensivePointsPerGame']['max'], defensivePointsPerGame)
        year_dict['defensivePointsPerGame']['min'] = min(year_dict['defensivePointsPerGame']['min'], defensivePointsPerGame)

        winPercentage = row['won'] / row['GP'] * 100

        year_dict.setdefault('winPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['winPercentage']['max'] = max(year_dict['winPercentage']['max'], winPercentage)
        year_dict['winPercentage']['min'] = min(year_dict['winPercentage']['min'], winPercentage)

        homeWinPercentage = row['homeW'] / (row['homeW'] + row['homeL']) * 100

        year_dict.setdefault('homeWinPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['homeWinPercentage']['max'] = max(year_dict['homeWinPercentage']['max'], homeWinPercentage)
        year_dict['homeWinPercentage']['min'] = min(year_dict['homeWinPercentage']['min'], homeWinPercentage)

        awayWinPercentage = row['awayW'] / (row['awayW'] + row['awayL']) * 100

        year_dict.setdefault('awayWinPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['awayWinPercentage']['max'] = max(year_dict['awayWinPercentage']['max'], awayWinPercentage)
        year_dict['awayWinPercentage']['min'] = min(year_dict['awayWinPercentage']['min'], awayWinPercentage)

        conferenceWinPercentage = row['confW'] / (row['confW'] + row['confL']) * 100

        year_dict.setdefault('conferenceWinPercentage', {'max': -float('inf'), 'min': float('inf')})
        year_dict['conferenceWinPercentage']['max'] = max(year_dict['conferenceWinPercentage']['max'], conferenceWinPercentage)
        year_dict['conferenceWinPercentage']['min'] = min(year_dict['conferenceWinPercentage']['min'], conferenceWinPercentage)

        attendance = row['attend']

        year_dict.setdefault('attendance', {'max': -float('inf'), 'min': float('inf')})
        year_dict['attendance']['max'] = max(year_dict['attendance']['max'], attendance)
        year_dict['attendance']['min'] = min(year_dict['attendance']['min'], attendance)

        rank = row['rank']

        year_dict.setdefault('rank', {'max': -float('inf'), 'min': float('inf')})
        year_dict['rank']['max'] = max(year_dict['rank']['max'], rank)
        year_dict['rank']['min'] = min(year_dict['rank']['min'], rank)

        o_oreb = row['o_oreb']

        year_dict.setdefault('o_oreb', {'max': -float('inf'), 'min': float('inf')})
        year_dict['o_oreb']['max'] = max(year_dict['o_oreb']['max'], o_oreb)
        year_dict['o_oreb']['min'] = min(year_dict['o_oreb']['min'], o_oreb)

        o_dreb = row['o_dreb']

        year_dict.setdefault('o_dreb', {'max': -float('inf'), 'min': float('inf')})
        year_dict['o_dreb']['max'] = max(year_dict['o_dreb']['max'], o_dreb)
        year_dict['o_dreb']['min'] = min(year_dict['o_dreb']['min'], o_dreb)

        d_oreb = row['d_oreb']

        year_dict.setdefault('d_oreb', {'max': -float('inf'), 'min': float('inf')})
        year_dict['d_oreb']['max'] = max(year_dict['d_oreb']['max'], d_oreb)
        year_dict['d_oreb']['min'] = min(year_dict['d_oreb']['min'], d_oreb)

        d_dreb = row['d_dreb']

        year_dict.setdefault('d_dreb', {'max': -float('inf'), 'min': float('inf')})
        year_dict['d_dreb']['max'] = max(year_dict['d_dreb']['max'], d_dreb)
        year_dict['d_dreb']['min'] = min(year_dict['d_dreb']['min'], d_dreb)

    yearly_stats[year] = year_dict

The team_power_ranking function is designed to calculate the power ranking for a given basketball team based on a set of offensive, defensive, and global statistics. It employs min-max scaling to normalize the statistics and then computes an average power ranking.

Power Ranking Calculation:
- Computes a weighted average of all relevant features.
- Uses an accumulator (powerRankingAcummulator) to sum up the weighted features.
- Returns a list containing the calculated power ranking followed by the relevant features.

In [ ]:
def team_power_ranking(team): 

    year = team['year']

    # offensive stats
    offensiveFieldGoalPercentage = (team['o_fgm'] / team['o_fga']) * 100
    offensiveFieldGoalPercentage = (offensiveFieldGoalPercentage - yearly_stats[year]['offensiveFieldGoalPercentage']['min']) / (
                yearly_stats[year]['offensiveFieldGoalPercentage']['max'] - yearly_stats[year]['offensiveFieldGoalPercentage']['min']) * 100

    offensiveFreeThrowPercentage = (team['o_ftm'] / team['o_fta']) * 100
    offensiveFreeThrowPercentage = (offensiveFreeThrowPercentage - yearly_stats[year]['offensiveFreeThrowPercentage']['min']) / (
                yearly_stats[year]['offensiveFreeThrowPercentage']['max'] - yearly_stats[year]['offensiveFreeThrowPercentage']['min']) * 100

    offensiveThreePointPercentage = (team['o_3pm'] / team['o_3pa']) * 100
    offensiveThreePointPercentage = (offensiveThreePointPercentage - yearly_stats[year]['offensiveThreePointPercentage']['min']) / (
                yearly_stats[year]['offensiveThreePointPercentage']['max'] - yearly_stats[year]['offensiveThreePointPercentage']['min']) * 100

    offensiveReboundsPerGame = team['o_reb'] / team['GP']
    offensiveReboundsPerGame = (offensiveReboundsPerGame - yearly_stats[year]['offensiveReboundsPerGame']['min']) / (
                yearly_stats[year]['offensiveReboundsPerGame']['max'] - yearly_stats[year]['offensiveReboundsPerGame']['min']) * 100

    offensiveAssistsPerGame = team['o_asts'] / team['GP']
    offensiveAssistsPerGame = (offensiveAssistsPerGame - yearly_stats[year]['offensiveAssistsPerGame']['min']) / (
                yearly_stats[year]['offensiveAssistsPerGame']['max'] - yearly_stats[year]['offensiveAssistsPerGame']['min']) * 100

    offensivePersonalFoulsPerGame = team['o_pf'] / team['GP']
    offensivePersonalFoulsPerGame = (offensivePersonalFoulsPerGame - yearly_stats[year]['offensivePersonalFoulsPerGame']['min']) / (
                yearly_stats[year]['offensivePersonalFoulsPerGame']['max'] - yearly_stats[year]['offensivePersonalFoulsPerGame']['min']) * 100

    offensiveStealsPerGame = team['o_stl'] / team['GP']
    offensiveStealsPerGame = (offensiveStealsPerGame - yearly_stats[year]['offensiveStealsPerGame']['min']) / (
                yearly_stats[year]['offensiveStealsPerGame']['max'] - yearly_stats[year]['offensiveStealsPerGame']['min']) * 100

    offensiveTurnoversPerGame = team['o_to'] / team['GP']
    offensiveTurnoversPerGame = (offensiveTurnoversPerGame - yearly_stats[year]['offensiveTurnoversPerGame']['min']) / (
                yearly_stats[year]['offensiveTurnoversPerGame']['max'] - yearly_stats[year]['offensiveTurnoversPerGame']['min']) * 100

    offensiveBlocksPerGame = team['o_blk'] / team['GP']
    offensiveBlocksPerGame = (offensiveBlocksPerGame - yearly_stats[year]['offensiveBlocksPerGame']['min']) / (
                yearly_stats[year]['offensiveBlocksPerGame']['max'] - yearly_stats[year]['offensiveBlocksPerGame']['min']) * 100

    offensivePointsPerGame = team['o_pts'] / team['GP']
    offensivePointsPerGame = (offensivePointsPerGame - yearly_stats[year]['offensivePointsPerGame']['min']) / (
                yearly_stats[year]['offensivePointsPerGame']['max'] - yearly_stats[year]['offensivePointsPerGame']['min']) * 100

    # defensive stats
    defensiveFieldGoalPercentage = (team['d_fgm'] / team['d_fga']) * 100
    defensiveFieldGoalPercentage = (defensiveFieldGoalPercentage - yearly_stats[year]['defensiveFieldGoalPercentage']['min']) / (
                yearly_stats[year]['defensiveFieldGoalPercentage']['max'] - yearly_stats[year]['defensiveFieldGoalPercentage']['min']) * 100

    defensiveFreeThrowPercentage = (team['d_ftm'] / team['d_fta']) * 100
    defensiveFreeThrowPercentage = (defensiveFreeThrowPercentage - yearly_stats[year]['defensiveFreeThrowPercentage']['min']) / (
                yearly_stats[year]['defensiveFreeThrowPercentage']['max'] - yearly_stats[year]['defensiveFreeThrowPercentage']['min']) * 100

    defensiveThreePointPercentage = (team['d_3pm'] / team['d_3pa']) * 100
    defensiveThreePointPercentage = (defensiveThreePointPercentage - yearly_stats[year]['defensiveThreePointPercentage']['min']) / (
                yearly_stats[year]['defensiveThreePointPercentage']['max'] - yearly_stats[year]['defensiveThreePointPercentage']['min']) * 100

    defensiveReboundsPerGame = team['d_reb'] / team['GP']
    defensiveReboundsPerGame = (defensiveReboundsPerGame - yearly_stats[year]['defensiveReboundsPerGame']['min']) / (
                yearly_stats[year]['defensiveReboundsPerGame']['max'] - yearly_stats[year]['defensiveReboundsPerGame']['min']) * 100

    defensiveAssistsPerGame = team['d_asts'] / team['GP']
    defensiveAssistsPerGame = (defensiveAssistsPerGame - yearly_stats[year]['defensiveAssistsPerGame']['min']) / (
                yearly_stats[year]['defensiveAssistsPerGame']['max'] - yearly_stats[year]['defensiveAssistsPerGame']['min']) * 100

    defensivePersonalFoulsPerGame = team['d_pf'] / team['GP']
    defensivePersonalFoulsPerGame = (defensivePersonalFoulsPerGame - yearly_stats[year]['defensivePersonalFoulsPerGame']['min']) / (
                yearly_stats[year]['defensivePersonalFoulsPerGame']['max'] - yearly_stats[year]['defensivePersonalFoulsPerGame']['min']) * 100

    defensiveStealsPerGame = team['d_stl'] / team['GP']
    defensiveStealsPerGame = (defensiveStealsPerGame - yearly_stats[year]['defensiveStealsPerGame']['min']) / (
                yearly_stats[year]['defensiveStealsPerGame']['max'] - yearly_stats[year]['defensiveStealsPerGame']['min']) * 100

    defensiveTurnoversPerGame = team['d_to'] / team['GP']
    defensiveTurnoversPerGame = (defensiveTurnoversPerGame - yearly_stats[year]['defensiveTurnoversPerGame']['min']) / (
                yearly_stats[year]['defensiveTurnoversPerGame']['max'] - yearly_stats[year]['defensiveTurnoversPerGame']['min']) * 100

    defensiveBlocksPerGame = team['d_blk'] / team['GP']
    defensiveBlocksPerGame = (defensiveBlocksPerGame - yearly_stats[year]['defensiveBlocksPerGame']['min']) / (
                yearly_stats[year]['defensiveBlocksPerGame']['max'] - yearly_stats[year]['defensiveBlocksPerGame']['min']) * 100

    defensivePointsPerGame = team['d_pts'] / team['GP']
    defensivePointsPerGame = (defensivePointsPerGame - yearly_stats[year]['defensivePointsPerGame']['min']) / (
                yearly_stats[year]['defensivePointsPerGame']['max'] - yearly_stats[year]['defensivePointsPerGame']['min']) * 100


    # global stats
    winPercentage = team['won'] / team['GP'] * 100
    winPercentage = (winPercentage - yearly_stats[year]['winPercentage']['min']) / (
                yearly_stats[year]['winPercentage']['max'] - yearly_stats[year]['winPercentage']['min']) * 100

    homeWinPercentage = team['homeW'] / (team['homeW'] + team['homeL']) * 100
    homeWinPercentage = (homeWinPercentage - yearly_stats[year]['homeWinPercentage']['min']) / (
                yearly_stats[year]['homeWinPercentage']['max'] - yearly_stats[year]['homeWinPercentage']['min']) * 100

    awayWinPercentage = team['awayW'] / (team['awayW'] + team['awayL']) * 100
    awayWinPercentage = (awayWinPercentage - yearly_stats[year]['awayWinPercentage']['min']) / (
                yearly_stats[year]['awayWinPercentage']['max'] - yearly_stats[year]['awayWinPercentage']['min']) * 100

    conferenceWinPercentage = team['confW'] / (team['confW'] + team['confL']) * 100
    conferenceWinPercentage = (conferenceWinPercentage - yearly_stats[year]['conferenceWinPercentage']['min']) / (
                yearly_stats[year]['conferenceWinPercentage']['max'] - yearly_stats[year]['conferenceWinPercentage']['min']) * 100

    attendance = (team['attend'] - yearly_stats[year]['attendance']['min']) / (
                yearly_stats[year]['attendance']['max'] - yearly_stats[year]['attendance']['min']) * 100

    relevantFeatures = [ offensiveFieldGoalPercentage, offensiveFreeThrowPercentage, offensiveThreePointPercentage, offensiveReboundsPerGame, offensiveAssistsPerGame, 
                        offensivePersonalFoulsPerGame, offensiveStealsPerGame, offensiveTurnoversPerGame, offensiveBlocksPerGame, offensivePointsPerGame, 
                        defensiveFieldGoalPercentage, defensiveFreeThrowPercentage, defensiveThreePointPercentage, defensiveReboundsPerGame, defensiveAssistsPerGame, 
                        defensivePersonalFoulsPerGame, defensiveStealsPerGame, defensiveTurnoversPerGame, defensiveBlocksPerGame, defensivePointsPerGame, winPercentage, 
                        homeWinPercentage, awayWinPercentage, conferenceWinPercentage, attendance ]
    
    powerRankingAcummulator = 0
    numberOfFeatures = len(relevantFeatures)
    averageFeatureWeight = 1 / numberOfFeatures

    for feature in relevantFeatures:
        powerRankingAcummulator += feature * averageFeatureWeight

    return [powerRankingAcummulator] + relevantFeatures

In [ ]:
team1 = teams.copy()

for index, team in team1.iterrows(): 
    powerRanking = team_power_ranking(team)

    team1.at[index, 'powerRanking'] = powerRanking[0]
    team1.at[index, 'offensiveFieldGoalPercentage'] = powerRanking[1]
    team1.at[index, 'offensiveFreeThrowPercentage'] = powerRanking[2]
    team1.at[index, 'offensiveThreePointPercentage'] = powerRanking[3]
    team1.at[index, 'offensiveReboundsPerGame'] = powerRanking[4]
    team1.at[index, 'offensiveAssistsPerGame'] = powerRanking[5]
    team1.at[index, 'offensivePersonalFoulsPerGame'] = powerRanking[6]
    team1.at[index, 'offensiveStealsPerGame'] = powerRanking[7]
    team1.at[index, 'offensiveTurnoversPerGame'] = powerRanking[8]
    team1.at[index, 'offensiveBlocksPerGame'] = powerRanking[9]
    team1.at[index, 'offensivePointsPerGame'] = powerRanking[10]
    team1.at[index, 'defensiveFieldGoalPercentage'] = powerRanking[11]
    team1.at[index, 'defensiveFreeThrowPercentage'] = powerRanking[12]
    team1.at[index, 'defensiveThreePointPercentage'] = powerRanking[13]
    team1.at[index, 'defensiveReboundsPerGame'] = powerRanking[14]
    team1.at[index, 'defensiveAssistsPerGame'] = powerRanking[15]
    team1.at[index, 'defensivePersonalFoulsPerGame'] = powerRanking[16]
    team1.at[index, 'defensiveStealsPerGame'] = powerRanking[17]
    team1.at[index, 'defensiveTurnoversPerGame'] = powerRanking[18]
    team1.at[index, 'defensiveBlocksPerGame'] = powerRanking[19]
    team1.at[index, 'defensivePointsPerGame'] = powerRanking[20]
    team1.at[index, 'winPercentage'] = powerRanking[21]
    team1.at[index, 'homeWinPercentage'] = powerRanking[22]
    team1.at[index, 'awayWinPercentage'] = powerRanking[23]
    team1.at[index, 'conferenceWinPercentage'] = powerRanking[24]
    team1.at[index, 'attendance'] = powerRanking[25]


team1 = team1.drop(columns=['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'won', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'attend'])
team1 = team1.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
team1.to_csv('team1.csv', index=False)

features_weight = correlation_matrix(team1)

In [ ]:
chi_square(team1, 'playoff')

In [ ]:
point_biserial(team1, ['powerRanking'], 'playoff')

This code down below creates a new dataset (``team1_orig``) by removing specific columns from the original team data (``teams_original``). The excluded columns are not so relevant anymore because the power ranking calculated before already has information relating all of them. 


In [ ]:
team1_orig = teams_original.drop(columns=['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'won', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'attend'])
team1_orig = teams_original.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
team1_orig = teams_original.drop(columns=['o_oreb', 'o_dreb', 'd_oreb', 'd_dreb', 'lost', 'min'])

team1_orig.to_csv('team1.csv', index=False)

This function calculates a team power ranking based on correlation matrix values and a set of customizable weights for various statistical features. This version provides a more balanced and nuanced assessment compared to the previous approach, allowing for greater flexibility in evaluating different aspects of the teams performance by considering the relationships between variables.

In [ ]:
def team_power_ranking_2(team, variables_weight): 
    
    year = team['year']

    offensiveFieldGoalPercentage = (team['o_fgm'] / team['o_fga']) * 100
    offensiveFieldGoalPercentage = (offensiveFieldGoalPercentage - yearly_stats[year]['offensiveFieldGoalPercentage']['min']) / (
                yearly_stats[year]['offensiveFieldGoalPercentage']['max'] - yearly_stats[year]['offensiveFieldGoalPercentage']['min']) * 100

    offensiveFreeThrowPercentage = (team['o_ftm'] / team['o_fta']) * 100
    offensiveFreeThrowPercentage = (offensiveFreeThrowPercentage - yearly_stats[year]['offensiveFreeThrowPercentage']['min']) / (
                yearly_stats[year]['offensiveFreeThrowPercentage']['max'] - yearly_stats[year]['offensiveFreeThrowPercentage']['min']) * 100

    offensiveThreePointPercentage = (team['o_3pm'] / team['o_3pa']) * 100
    offensiveThreePointPercentage = (offensiveThreePointPercentage - yearly_stats[year]['offensiveThreePointPercentage']['min']) / (
                yearly_stats[year]['offensiveThreePointPercentage']['max'] - yearly_stats[year]['offensiveThreePointPercentage']['min']) * 100

    offensiveReboundsPerGame = team['o_reb'] / team['GP']
    offensiveReboundsPerGame = (offensiveReboundsPerGame - yearly_stats[year]['offensiveReboundsPerGame']['min']) / (
                yearly_stats[year]['offensiveReboundsPerGame']['max'] - yearly_stats[year]['offensiveReboundsPerGame']['min']) * 100

    offensiveAssistsPerGame = team['o_asts'] / team['GP']
    offensiveAssistsPerGame = (offensiveAssistsPerGame - yearly_stats[year]['offensiveAssistsPerGame']['min']) / (
                yearly_stats[year]['offensiveAssistsPerGame']['max'] - yearly_stats[year]['offensiveAssistsPerGame']['min']) * 100

    offensivePersonalFoulsPerGame = team['o_pf'] / team['GP']
    offensivePersonalFoulsPerGame = (offensivePersonalFoulsPerGame - yearly_stats[year]['offensivePersonalFoulsPerGame']['min']) / (
                yearly_stats[year]['offensivePersonalFoulsPerGame']['max'] - yearly_stats[year]['offensivePersonalFoulsPerGame']['min']) * 100

    offensiveStealsPerGame = team['o_stl'] / team['GP']
    offensiveStealsPerGame = (offensiveStealsPerGame - yearly_stats[year]['offensiveStealsPerGame']['min']) / (
                yearly_stats[year]['offensiveStealsPerGame']['max'] - yearly_stats[year]['offensiveStealsPerGame']['min']) * 100

    offensiveTurnoversPerGame = team['o_to'] / team['GP']
    offensiveTurnoversPerGame = (offensiveTurnoversPerGame - yearly_stats[year]['offensiveTurnoversPerGame']['min']) / (
                yearly_stats[year]['offensiveTurnoversPerGame']['max'] - yearly_stats[year]['offensiveTurnoversPerGame']['min']) * 100

    offensiveBlocksPerGame = team['o_blk'] / team['GP']
    offensiveBlocksPerGame = (offensiveBlocksPerGame - yearly_stats[year]['offensiveBlocksPerGame']['min']) / (
                yearly_stats[year]['offensiveBlocksPerGame']['max'] - yearly_stats[year]['offensiveBlocksPerGame']['min']) * 100

    offensivePointsPerGame = team['o_pts'] / team['GP']
    offensivePointsPerGame = (offensivePointsPerGame - yearly_stats[year]['offensivePointsPerGame']['min']) / (
                yearly_stats[year]['offensivePointsPerGame']['max'] - yearly_stats[year]['offensivePointsPerGame']['min']) * 100

    # defensive stats
    defensiveFieldGoalPercentage = (team['d_fgm'] / team['d_fga']) * 100
    defensiveFieldGoalPercentage = (defensiveFieldGoalPercentage - yearly_stats[year]['defensiveFieldGoalPercentage']['min']) / (
                yearly_stats[year]['defensiveFieldGoalPercentage']['max'] - yearly_stats[year]['defensiveFieldGoalPercentage']['min']) * 100

    defensiveFreeThrowPercentage = (team['d_ftm'] / team['d_fta']) * 100
    defensiveFreeThrowPercentage = (defensiveFreeThrowPercentage - yearly_stats[year]['defensiveFreeThrowPercentage']['min']) / (
                yearly_stats[year]['defensiveFreeThrowPercentage']['max'] - yearly_stats[year]['defensiveFreeThrowPercentage']['min']) * 100

    defensiveThreePointPercentage = (team['d_3pm'] / team['d_3pa']) * 100
    defensiveThreePointPercentage = (defensiveThreePointPercentage - yearly_stats[year]['defensiveThreePointPercentage']['min']) / (
                yearly_stats[year]['defensiveThreePointPercentage']['max'] - yearly_stats[year]['defensiveThreePointPercentage']['min']) * 100

    defensiveReboundsPerGame = team['d_reb'] / team['GP']
    defensiveReboundsPerGame = (defensiveReboundsPerGame - yearly_stats[year]['defensiveReboundsPerGame']['min']) / (
                yearly_stats[year]['defensiveReboundsPerGame']['max'] - yearly_stats[year]['defensiveReboundsPerGame']['min']) * 100

    defensiveAssistsPerGame = team['d_asts'] / team['GP']
    defensiveAssistsPerGame = (defensiveAssistsPerGame - yearly_stats[year]['defensiveAssistsPerGame']['min']) / (
                yearly_stats[year]['defensiveAssistsPerGame']['max'] - yearly_stats[year]['defensiveAssistsPerGame']['min']) * 100

    defensivePersonalFoulsPerGame = team['d_pf'] / team['GP']
    defensivePersonalFoulsPerGame = (defensivePersonalFoulsPerGame - yearly_stats[year]['defensivePersonalFoulsPerGame']['min']) / (
                yearly_stats[year]['defensivePersonalFoulsPerGame']['max'] - yearly_stats[year]['defensivePersonalFoulsPerGame']['min']) * 100

    defensiveStealsPerGame = team['d_stl'] / team['GP']
    defensiveStealsPerGame = (defensiveStealsPerGame - yearly_stats[year]['defensiveStealsPerGame']['min']) / (
                yearly_stats[year]['defensiveStealsPerGame']['max'] - yearly_stats[year]['defensiveStealsPerGame']['min']) * 100

    defensiveTurnoversPerGame = team['d_to'] / team['GP']
    defensiveTurnoversPerGame = (defensiveTurnoversPerGame - yearly_stats[year]['defensiveTurnoversPerGame']['min']) / (
                yearly_stats[year]['defensiveTurnoversPerGame']['max'] - yearly_stats[year]['defensiveTurnoversPerGame']['min']) * 100

    defensiveBlocksPerGame = team['d_blk'] / team['GP']
    defensiveBlocksPerGame = (defensiveBlocksPerGame - yearly_stats[year]['defensiveBlocksPerGame']['min']) / (
                yearly_stats[year]['defensiveBlocksPerGame']['max'] - yearly_stats[year]['defensiveBlocksPerGame']['min']) * 100

    defensivePointsPerGame = team['d_pts'] / team['GP']
    defensivePointsPerGame = (defensivePointsPerGame - yearly_stats[year]['defensivePointsPerGame']['min']) / (
                yearly_stats[year]['defensivePointsPerGame']['max'] - yearly_stats[year]['defensivePointsPerGame']['min']) * 100


    # global stats
    winPercentage = team['won'] / team['GP'] * 100
    winPercentage = (winPercentage - yearly_stats[year]['winPercentage']['min']) / (
                yearly_stats[year]['winPercentage']['max'] - yearly_stats[year]['winPercentage']['min']) * 100

    homeWinPercentage = team['homeW'] / (team['homeW'] + team['homeL']) * 100
    homeWinPercentage = (homeWinPercentage - yearly_stats[year]['homeWinPercentage']['min']) / (
                yearly_stats[year]['homeWinPercentage']['max'] - yearly_stats[year]['homeWinPercentage']['min']) * 100

    awayWinPercentage = team['awayW'] / (team['awayW'] + team['awayL']) * 100
    awayWinPercentage = (awayWinPercentage - yearly_stats[year]['awayWinPercentage']['min']) / (
                yearly_stats[year]['awayWinPercentage']['max'] - yearly_stats[year]['awayWinPercentage']['min']) * 100

    conferenceWinPercentage = team['confW'] / (team['confW'] + team['confL']) * 100
    conferenceWinPercentage = (conferenceWinPercentage - yearly_stats[year]['conferenceWinPercentage']['min']) / (
                yearly_stats[year]['conferenceWinPercentage']['max'] - yearly_stats[year]['conferenceWinPercentage']['min']) * 100

    attendance = (team['attend'] - yearly_stats[year]['attendance']['min']) / (
                yearly_stats[year]['attendance']['max'] - yearly_stats[year]['attendance']['min']) * 100

    firstRoundValue = 0
    semisValue = 0
    finalsValue = 0

    if team['firstRound'] == '' or team['firstRound'] is None or team['firstRound'] == -1.0: 
        firstRoundValue = 0
        semisValue = 0
        finalsValue = 0
    elif team['firstRound'] == 'L' or team['firstRound'] == 0.0:
        firstRoundValue = 5
        semisValue = 0
        finalsValue = 0
    else: 
        if team['semis'] == '' or team['semis'] is None or team['semis'] == -1.0: 
            firstRoundValue = 10
            semisValue = 0
            finalsValue = 0
        elif team['semis'] == 'L' or team['semis'] == 0.0:
            firstRoundValue = 10
            semisValue = 5
            finalsValue = 0
        else: 
            if team['finals'] == '' or team['finals'] is None or team['finals'] == -1.0: 
                firstRoundValue = 10
                semisValue = 10
                finalsValue = 0
            elif team['finals'] == 'L' or team['finals'] == 0.0:
                firstRoundValue = 10
                semisValue = 10
                finalsValue = 5
            else: 
                firstRoundValue = 10
                semisValue = 10
                finalsValue = 10

    rankValue = rankMax / team['rank'] 

    o_orebPercentage = (team['o_oreb'] - o_orebMin) / (o_orebMax - o_orebMin) * 100

    o_drebPercentage = (team['o_dreb'] - o_drebMin) / (o_drebMax - o_drebMin) * 100

    d_orebPercentage = (team['d_oreb'] - d_orebMin) / (d_orebMax - d_orebMin) * 100

    d_drebPercentage = (team['d_dreb'] - d_drebMin) / (d_drebMax - d_drebMin) * 100

    relevantFeatures = {'offensiveFieldGoalPercentage': offensiveFieldGoalPercentage,
        'offensiveFreeThrowPercentage': offensiveFreeThrowPercentage,
        'offensiveThreePointPercentage': offensiveThreePointPercentage,
        'offensiveReboundsPerGame': offensiveReboundsPerGame,
        'offensiveAssistsPerGame': offensiveAssistsPerGame,
        'offensivePersonalFoulsPerGame': offensivePersonalFoulsPerGame,
        'offensiveStealsPerGame': offensiveStealsPerGame,
        'offensiveTurnoversPerGame': offensiveTurnoversPerGame,
        'offensiveBlocksPerGame': offensiveBlocksPerGame,
        'offensivePointsPerGame': offensivePointsPerGame,
        'defensiveFieldGoalPercentage': defensiveFieldGoalPercentage,
        'defensiveFreeThrowPercentage': defensiveFreeThrowPercentage,
        'defensiveThreePointPercentage': defensiveThreePointPercentage,
        'defensiveReboundsPerGame': defensiveReboundsPerGame,
        'defensiveAssistsPerGame': defensiveAssistsPerGame,
        'defensivePersonalFoulsPerGame': defensivePersonalFoulsPerGame,
        'defensiveStealsPerGame': defensiveStealsPerGame,
        'defensiveTurnoversPerGame': defensiveTurnoversPerGame,
        'defensiveBlocksPerGame': defensiveBlocksPerGame,
        'defensivePointsPerGame': defensivePointsPerGame,
        'winPercentage': winPercentage,
        'homeWinPercentage': homeWinPercentage,
        'awayWinPercentage': awayWinPercentage,
        'conferenceWinPercentage': conferenceWinPercentage,
        'attendance': attendance,
        'firstRound': firstRoundValue,
        'semis': semisValue,
        'finals': finalsValue, 
        'rank': rankValue, 
        'o_oreb': o_orebPercentage,
        'o_dreb': o_drebPercentage,
        'd_oreb': d_orebPercentage,
        'd_dreb': d_drebPercentage
    }

    powerRankingAccumulator = 0

    for feature_name, feature_value in relevantFeatures.items():
        powerRankingAccumulator += feature_value * float(variables_weight[feature_name])

    relevantFeatures['powerRanking2'] = powerRankingAccumulator

    print(relevantFeatures)

    return relevantFeatures

The process involves: 

- Iterate through each team in the dataset (team2).
- Calculate a power ranking for each team using the ``team_power_ranking_2`` funtion.
- Updates the team2 DataFrame with the calculated metrics for each team.
- Removes redundant or unnecessary columns from the DataFrame to streamline and focus on relevant features.
- Generates a new correlation matrix (features_weight) based on the updated team2 DataFrame, which will be used in subsequent analyses.

In [ ]:
team2 = teams.copy()

for index, team in team2.iterrows(): 
    powerRanking = team_power_ranking_2(team, features_weight)

    team2.at[index, 'powerRanking2'] = powerRanking['powerRanking2']
    team2.at[index, 'offensiveFieldGoalPercentage'] = powerRanking['offensiveFieldGoalPercentage']
    team2.at[index, 'offensiveFreeThrowPercentage'] = powerRanking['offensiveFreeThrowPercentage']
    team2.at[index, 'offensiveThreePointPercentage'] = powerRanking['offensiveThreePointPercentage']
    team2.at[index, 'offensiveReboundsPerGame'] = powerRanking['offensiveReboundsPerGame']
    team2.at[index, 'offensiveAssistsPerGame'] = powerRanking['offensiveAssistsPerGame']
    team2.at[index, 'offensivePersonalFoulsPerGame'] = powerRanking['offensivePersonalFoulsPerGame']
    team2.at[index, 'offensiveStealsPerGame'] = powerRanking['offensiveStealsPerGame']
    team2.at[index, 'offensiveTurnoversPerGame'] = powerRanking['offensiveTurnoversPerGame']
    team2.at[index, 'offensiveBlocksPerGame'] = powerRanking['offensiveBlocksPerGame']
    team2.at[index, 'offensivePointsPerGame'] = powerRanking['offensivePointsPerGame']
    team2.at[index, 'defensiveFieldGoalPercentage'] = powerRanking['defensiveFieldGoalPercentage']
    team2.at[index, 'defensiveFreeThrowPercentage'] = powerRanking['defensiveFreeThrowPercentage']
    team2.at[index, 'defensiveThreePointPercentage'] = powerRanking['defensiveThreePointPercentage']
    team2.at[index, 'defensiveReboundsPerGame'] = powerRanking['defensiveReboundsPerGame']
    team2.at[index, 'defensiveAssistsPerGame'] = powerRanking['defensiveAssistsPerGame']
    team2.at[index, 'defensivePersonalFoulsPerGame'] = powerRanking['defensivePersonalFoulsPerGame']
    team2.at[index, 'defensiveStealsPerGame'] = powerRanking['defensiveStealsPerGame']
    team2.at[index, 'defensiveTurnoversPerGame'] = powerRanking['defensiveTurnoversPerGame']
    team2.at[index, 'defensiveBlocksPerGame'] = powerRanking['defensiveBlocksPerGame']
    team2.at[index, 'defensivePointsPerGame'] = powerRanking['defensivePointsPerGame']
    team2.at[index, 'winPercentage'] = powerRanking['winPercentage']
    team2.at[index, 'homeWinPercentage'] = powerRanking['homeWinPercentage']
    team2.at[index, 'awayWinPercentage'] = powerRanking['awayWinPercentage']
    team2.at[index, 'conferenceWinPercentage'] = powerRanking['conferenceWinPercentage']
    team2.at[index, 'attendance'] = powerRanking['attendance']
    team2.at[index, 'firstRound'] = powerRanking['firstRound']
    team2.at[index, 'semis'] = powerRanking['semis']
    team2.at[index, 'finals'] = powerRanking['finals']
    team2.at[index, 'rank'] = powerRanking['rank']
    team2.at[index, 'o_oreb'] = powerRanking['o_oreb']
    team2.at[index, 'o_dreb'] = powerRanking['o_dreb']
    team2.at[index, 'd_oreb'] = powerRanking['d_oreb']
    team2.at[index, 'd_dreb'] = powerRanking['d_dreb']


team2 = team2.drop(columns=['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'won', 'lost', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'attend'])
team2 = team2.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])

features_weight = correlation_matrix(team2)

In this section we do the same as we done before but specifically applied to the original dataset (``teams_original``). It calculates power rankings based on correlation matrix values and updates the original dataset with the computed metrics. The unnecessary or redundant columns are then removed for further analysis.

In [ ]:
for index, team in teams_original.iterrows(): 
    powerRanking = team_power_ranking_2(team, features_weight)

    teams_original.at[index, 'powerRanking2'] = powerRanking['powerRanking2']
    teams_original.at[index, 'offensiveFieldGoalPercentage'] = powerRanking['offensiveFieldGoalPercentage']
    teams_original.at[index, 'offensiveFreeThrowPercentage'] = powerRanking['offensiveFreeThrowPercentage']
    teams_original.at[index, 'offensiveThreePointPercentage'] = powerRanking['offensiveThreePointPercentage']
    teams_original.at[index, 'offensiveReboundsPerGame'] = powerRanking['offensiveReboundsPerGame']
    teams_original.at[index, 'offensiveAssistsPerGame'] = powerRanking['offensiveAssistsPerGame']
    teams_original.at[index, 'offensivePersonalFoulsPerGame'] = powerRanking['offensivePersonalFoulsPerGame']
    teams_original.at[index, 'offensiveStealsPerGame'] = powerRanking['offensiveStealsPerGame']
    teams_original.at[index, 'offensiveTurnoversPerGame'] = powerRanking['offensiveTurnoversPerGame']
    teams_original.at[index, 'offensiveBlocksPerGame'] = powerRanking['offensiveBlocksPerGame']
    teams_original.at[index, 'offensivePointsPerGame'] = powerRanking['offensivePointsPerGame']
    teams_original.at[index, 'defensiveFieldGoalPercentage'] = powerRanking['defensiveFieldGoalPercentage']
    teams_original.at[index, 'defensiveFreeThrowPercentage'] = powerRanking['defensiveFreeThrowPercentage']
    teams_original.at[index, 'defensiveThreePointPercentage'] = powerRanking['defensiveThreePointPercentage']
    teams_original.at[index, 'defensiveReboundsPerGame'] = powerRanking['defensiveReboundsPerGame']
    teams_original.at[index, 'defensiveAssistsPerGame'] = powerRanking['defensiveAssistsPerGame']
    teams_original.at[index, 'defensivePersonalFoulsPerGame'] = powerRanking['defensivePersonalFoulsPerGame']
    teams_original.at[index, 'defensiveStealsPerGame'] = powerRanking['defensiveStealsPerGame']
    teams_original.at[index, 'defensiveTurnoversPerGame'] = powerRanking['defensiveTurnoversPerGame']
    teams_original.at[index, 'defensiveBlocksPerGame'] = powerRanking['defensiveBlocksPerGame']
    teams_original.at[index, 'defensivePointsPerGame'] = powerRanking['defensivePointsPerGame']
    teams_original.at[index, 'winPercentage'] = powerRanking['winPercentage']
    teams_original.at[index, 'homeWinPercentage'] = powerRanking['homeWinPercentage']
    teams_original.at[index, 'awayWinPercentage'] = powerRanking['awayWinPercentage']
    teams_original.at[index, 'conferenceWinPercentage'] = powerRanking['conferenceWinPercentage']
    teams_original.at[index, 'attendance'] = powerRanking['attendance']


teams_original = teams_original.drop(columns=['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'won', 'lost', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'attend'])
teams_original = teams_original.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])

Now we have a much shorter dataframe, but as we can see the relation between minutes and games played is really big, so we can delete games played. 

In [ ]:
team2 = team2.drop('GP', axis=1)
teams_original = teams_original.drop('GP', axis=1)
#team2.to_csv('filtered/team2.csv', index=False)
correlation_matrix(team2)

teams_original.to_csv('filtered/team2.csv', index=False)

In [ ]:
chi_square(team2, 'playoff')

In [ ]:
point_biserial(team2, team2.columns, 'playoff')

In [ ]:
team2.to_csv('team2.csv', index=False)

#team2 = shift_target_variable(team2)

point_biserial(team2, ['powerRanking2'], 'playoff')

In [ ]:
teams_original['firstRound'] = team2['firstRound']
teams_original['semis'] = team2['semis']
teams_original['finals'] = team2['finals']
teams_original['powerRanking2'] = team2['powerRanking2']

teams_original.drop(columns=['o_oreb', 'd_oreb', 'd_dreb', 'min', 'offensiveFreeThrowPercentage', 'offensiveStealsPerGame', 'offensiveTurnoversPerGame', 
                    'defensiveFreeThrowPercentage', 'defensivePersonalFoulsPerGame', 'defensiveStealsPerGame', 'defensiveTurnoversPerGame', 'arena'], inplace=True)

teams_original.to_csv('filtered/team2_before_shift.csv', index=False)

teams_original = shift_target_variable(teams_original)

teams_original.to_csv('filtered/team2.csv', index=False)